In [120]:
import mxnet as mx
import numpy as np
import scipy as sp
import sys
import networkx as nx
import igraph as ig
import logging
import time

In [117]:
class AutoEncoderModel:
    def __init__(self, dims, internal_act=None, output_act=None):
        self.data = mx.symbol.Variable('data')
        self.y = mx.symbol.Variable('label')
        self.fc1_weight = mx.symbol.Variable('fc1_weight')
        self.fc1_bias = mx.symbol.Variable('fc1_bias')
        self.fc2_weight = mx.symbol.Variable('fc2_weight')
        self.fc2_bias = mx.symbol.Variable('fc2_bias')
        x = mx.symbol.FullyConnected(data=self.data, weight=self.fc1_weight,
                                     bias=self.fc1_bias, num_hidden=dims[1])
        if (internal_act is not None):
            x = mx.symbol.Activation(data=x, act_type=internal_act)
            print("Internal activation: " + internal_act)
        self.layer1 = x
        x = mx.symbol.FullyConnected(data=x, weight=self.fc2_weight,
                                     bias=self.fc2_bias, num_hidden=dims[2])
        if (output_act is not None):
            x = mx.symbol.Activation(data=x, act_type=output_act)
            print("Output activation: " + output_act)
        self.layer2 = x
        # TODO How about using L1/L2 regularization.
        self.loss = mx.symbol.LinearRegressionOutput(data=x, label=self.y)
        self.model = mx.mod.Module(symbol=self.loss, data_names=['data'], label_names = ['label'])

    def fit(self, data, batch_size, num_epoch, params=None, learning_rate=0.005, reinit_opt=True):
        data_iter = mx.io.NDArrayIter(data={'data':data}, label={'label':data},
                batch_size=batch_size, shuffle=True,
                last_batch_handle='roll_over')
        
        if (params is None):
            print("Learning rate: " + str(learning_rate))
            print("batch size: " + str(batch_size))
            print("internal #epochs: " + str(num_epoch))
            # allocate memory given the input data and label shapes
            self.model.bind(data_shapes=data_iter.provide_data, label_shapes=data_iter.provide_label)
            # initialize parameters by uniform random numbers
            self.model.init_params(initializer=mx.init.Uniform(scale=.1))
            # use SGD with learning rate 0.1 to train
            self.model.init_optimizer(optimizer='sgd',
                                      optimizer_params={'learning_rate': learning_rate,
                                                        'momentum': 0.9})
        else:
            self.model.set_params(arg_params=params, aux_params=None, force_init=True)
            if (reinit_opt):
                print("reinit optimizer. New learning rate: " + str(learning_rate))
                self.model.init_optimizer(optimizer='sgd',
                                          optimizer_params={'learning_rate': learning_rate,
                                                            'momentum': 0.9}, force_init=True)
        # use accuracy as the metric
        metric = mx.metric.create('acc')
        # train 5 epochs, i.e. going over the data iter one pass
        for epoch in range(num_epoch):
            data_iter.reset()
            metric.reset()
            for batch in data_iter:
                self.model.forward(batch, is_train=True)       # compute predictions
                self.model.update_metric(metric, batch.label)  # accumulate prediction accuracy
                self.model.backward()                          # compute gradients
                self.model.update()                            # update parameters
            #print('Epoch %d, Training %s' % (epoch, metric.get()))
        #self.model.fit(data_iter, optimizer_params={'learning_rate':learning_rate, 'momentum': 0.9},
        #        optimizer='sgd', num_epoch=50, eval_metric='mse', force_rebind=True,
        #        batch_end_callback = mx.callback.Speedometer(batch_size, 2))

In [133]:
def get_act(act):
    if (act == 'sigmoid'):
        return sp.special.expit
    elif (act == 'tanh'):
        return np.tanh
    elif (act == 'relu'):
        return lambda x: np.maximum(x, 0)
    else:
        return None

def train(data, num_dims, num_epoc, internal_act=None, output_act=None, learning_rate=0.005, batch_size=50):
    int_epoc = 100
    params = None
    model = AutoEncoderModel([data.shape[1], num_dims, data.shape[1]],
                             internal_act, output_act)
    prev_val = None
    reinit_opt = True
    for i in range(num_epoc/int_epoc):
        curr = time.time()
        model.fit(data, batch_size, int_epoc, params, learning_rate, reinit_opt=reinit_opt)
        print(str(int_epoc) + " epochs takes " + str(time.time() - curr) + " seconds")
        reinit_opt = False

        params = model.model.get_params()[0]
        fc1_weight = params.get('fc1_weight').asnumpy()
        fc1_bias = params.get('fc1_bias').asnumpy()
        fc2_weight = params.get('fc2_weight').asnumpy()
        fc2_bias = params.get('fc2_bias').asnumpy()

        np_data = data.asnumpy()
        hidden = np.dot(np_data, fc1_weight.T) + fc1_bias
        act_func = get_act(internal_act)
        if (act_func is not None):
            hidden = act_func(hidden)
        output = np.dot(hidden, fc2_weight.T) + fc2_bias
        act_func = get_act(output_act)
        if (act_func is not None):
            output = act_func(output)
        val = np.sum(np.square(output - np_data))
        print("epoc " + str((i + 1) * int_epoc) + ": " + str(val))
        if (prev_val is not None and prev_val < val):
            learning_rate = learning_rate / 2
            reinit_opt = True
        prev_val = val
        sys.stdout.flush()
    return params

## Run on a low-rank data

In [144]:
rand_data1 = mx.ndarray.random_uniform(shape=[1000, 10])
rand_data2 = mx.ndarray.random_uniform(shape=[10, 100])
rand_data = mx.ndarray.dot(rand_data1, rand_data2)
print("max: " + str(mx.ndarray.max(rand_data)))
rand_data = rand_data / mx.ndarray.max(rand_data)
print(rand_data.shape)

max: 
[ 5.70393372]
<NDArray 1 @cpu(0)>
(1000L, 100L)


In [145]:
np_rand_data = rand_data.asnumpy()
U, s, Vh = sp.sparse.linalg.svds(np_rand_data, k=10)
low_dim_data = np.dot(np_rand_data, Vh.T)
print(low_dim_data)
print(sum(low_dim_data[low_dim_data > 0]))
print(sum(low_dim_data[low_dim_data < 0]))
res = np.dot(low_dim_data, Vh)
print("svd error: " + str(np.sum(np.square(res - np_rand_data))))

[[-0.27671653  0.11496934  0.22401108 ..., -0.1150435   0.24723341
   4.58532953]
 [-0.05978573 -0.19632138 -0.02645103 ..., -0.28306383 -0.10512527
   5.29204845]
 [ 0.07837401  0.06769264  0.27338678 ...,  0.01698423  0.0203772
   3.47058678]
 ..., 
 [ 0.02188545  0.09232116  0.26843369 ...,  0.08963827  0.07680167
   3.58912826]
 [ 0.15893728 -0.1496805   0.11094719 ...,  0.2932995  -0.05907928
   4.64020538]
 [ 0.03004075  0.03486993  0.02080921 ...,  0.05377063  0.13145779
   4.39051485]]
4984.47589737
-528.592736117
svd error: 5.74806e-09


In [146]:
params_linear_r10=train(rand_data, 10, 5000, learning_rate=0.2, batch_size=50)

Learning rate: 0.2
batch size: 50
internal #epochs: 100
100 epochs takes 2.23493695259 seconds
epoc 0: 173.404
100 epochs takes 2.16981911659 seconds
epoc 100: 127.83
100 epochs takes 2.17004418373 seconds
epoc 200: 81.2349
100 epochs takes 2.26216101646 seconds
epoc 300: 49.3242
100 epochs takes 2.21550703049 seconds
epoc 400: 31.235
100 epochs takes 2.20563292503 seconds
epoc 500: 19.9435
100 epochs takes 2.22947406769 seconds
epoc 600: 13.2789
100 epochs takes 2.19789195061 seconds
epoc 700: 9.33316
100 epochs takes 2.29411387444 seconds
epoc 800: 6.16517
100 epochs takes 2.25340390205 seconds
epoc 900: 3.44699
100 epochs takes 2.25359797478 seconds
epoc 1000: 1.62342
100 epochs takes 2.22027087212 seconds
epoc 1100: 0.696142
100 epochs takes 2.18843913078 seconds
epoc 1200: 0.294522
100 epochs takes 2.2648499012 seconds
epoc 1300: 0.128039
100 epochs takes 2.2517490387 seconds
epoc 1400: 0.0572501
100 epochs takes 2.23173594475 seconds
epoc 1500: 0.026083
100 epochs takes 2.2771070

In [ ]:
params_sigmoid_r10=train(rand_data, 10, 5000, internal_act='tanh', learning_rate=0.4, batch_size=100)

Internal activation: tanh
Learning rate: 0.4
batch size: 100
internal #epochs: 100
100 epochs takes 1.3073489666 seconds
epoc 0: 184.826
100 epochs takes 1.33411216736 seconds
epoc 100: 151.107
100 epochs takes 1.3682179451 seconds
epoc 200: 104.392
100 epochs takes 1.34697604179 seconds
epoc 300: 66.445
100 epochs takes 1.3471288681 seconds
epoc 400: 42.5984
100 epochs takes 1.27479887009 seconds
epoc 500: 30.4482
100 epochs takes 1.34358310699 seconds
epoc 600: 23.3595
100 epochs takes 1.2657148838 seconds
epoc 700: 17.126
100 epochs takes 1.33024120331 seconds
epoc 800: 12.4677
100 epochs takes 1.3495721817 seconds
epoc 900: 9.29682
100 epochs takes 1.42844581604 seconds
epoc 1000: 6.61072
100 epochs takes 1.3458750248 seconds
epoc 1100: 4.28825
100 epochs takes 1.30867600441 seconds
epoc 1200: 2.67795
100 epochs takes 1.3676970005 seconds
epoc 1300: 1.80303
100 epochs takes 1.32587099075 seconds
epoc 1400: 1.38844
100 epochs takes 1.3152859211 seconds
epoc 1500: 1.18517
100 epochs 

## Run on real data

We compute the embedding on a graph with 81306 vertices and 1768149 vertices. To embed the graph into 10 dimensions, we start with the most densest columns and increase the number of columns to embed. When we increase the number of columns to embed, we use the parameters trained from the previous run (on the dataset with a smaller number of columns).

In [110]:
elg = nx.read_edgelist("/home/ubuntu/datasets/twitter_combined.txt")
spm = nx.to_scipy_sparse_matrix(elg, dtype='f')
colsum = np.ravel(spm.sum(axis=1))

### Compute the embedding on the densest 10 columns.

In [39]:
max10 = np.sort(np.ravel(colsum), axis=None)[len(colsum) - 10]
data10 = mx.ndarray.sparse.csr_matrix(spm[:,colsum >= max10])
print(mx.ndarray.sum(data10, axis=0))
print(data10.shape)


[ 2490.  3383.  2484.  2758.  2476.  1789.  2133.  3011.  3239.  2155.]
<NDArray 10 @cpu(0)>
(81306L, 10L)


In [114]:
np_data10 = data10.asnumpy()
U, s, Vh = sp.linalg.svd(np_data10, full_matrices=False)
low_dim_data = np.dot(np_data10, Vh.T)
print(low_dim_data.shape)
print(np.max(low_dim_data))
print(np.min(low_dim_data))
res = np.dot(low_dim_data, Vh)
print("svd error: " + str(np.sum(np.square(res - np_data10))))

(81306, 10)
1.21314
-2.3992
svd error: 6.78263e-09


In [143]:
params_linear10=train(data10, 10, 2000, internal_act=None, learning_rate=0.1, batch_size=2000)

Learning rate: 0.1
batch size: 2000
internal #epochs: 100
100 epochs takes 6.57384419441 seconds
epoc 0: 661.281
100 epochs takes 7.22010612488 seconds
epoc 100: 205.035
100 epochs takes 6.69524002075 seconds
epoc 200: 86.0281
100 epochs takes 6.86633992195 seconds
epoc 300: 69.2754
100 epochs takes 7.26551198959 seconds
epoc 400: 48.0851
100 epochs takes 7.08142089844 seconds
epoc 500: 24.3964
100 epochs takes 6.72911000252 seconds
epoc 600: 8.38904
100 epochs takes 6.66133213043 seconds
epoc 700: 2.06232
100 epochs takes 6.76626992226 seconds
epoc 800: 0.410246
100 epochs takes 7.24766898155 seconds
epoc 900: 0.0735736
100 epochs takes 6.81751203537 seconds
epoc 1000: 0.0125639
100 epochs takes 7.10305809975 seconds
epoc 1100: 0.00211213
100 epochs takes 7.08962607384 seconds
epoc 1200: 0.00035564
100 epochs takes 6.72433185577 seconds
epoc 1300: 6.57533e-05
100 epochs takes 6.78700900078 seconds
epoc 1400: 1.69345e-05
100 epochs takes 6.82878088951 seconds
epoc 1500: 1.13987e-05
100

In [140]:
params_sigmoid10=train(data10, 10, 10000, internal_act='tanh', learning_rate=0.1, batch_size=2000)

Internal activation: tanh
Learning rate: 0.1
batch size: 2000
internal #epochs: 100
100 epochs takes 8.79215788841 seconds
epoc 0: 782.973
100 epochs takes 8.54049897194 seconds
epoc 100: 365.227
100 epochs takes 8.9551320076 seconds
epoc 200: 177.58
100 epochs takes 8.56133294106 seconds
epoc 300: 121.924
100 epochs takes 8.85677790642 seconds
epoc 400: 115.84
100 epochs takes 8.66231513023 seconds
epoc 500: 110.476
100 epochs takes 8.81901407242 seconds
epoc 600: 102.967
100 epochs takes 9.03215098381 seconds
epoc 700: 91.0163
100 epochs takes 8.80674505234 seconds
epoc 800: 72.9704
100 epochs takes 8.79361891747 seconds
epoc 900: 51.2826
100 epochs takes 8.95192408562 seconds
epoc 1000: 33.1414
100 epochs takes 8.9709789753 seconds
epoc 1100: 22.8834
100 epochs takes 8.84673404694 seconds
epoc 1200: 18.4841
100 epochs takes 9.01217198372 seconds
epoc 1300: 16.636
100 epochs takes 8.86660695076 seconds
epoc 1400: 15.6374
100 epochs takes 8.89191913605 seconds
epoc 1500: 14.9088
100 e

In [141]:
params_sigmoid10=train(data10, 10, 10000, internal_act='tanh', output_act='sigmoid', learning_rate=0.2, batch_size=2000)

Internal activation: tanh
Output activation: sigmoid
Learning rate: 0.2
batch size: 2000
internal #epochs: 100
100 epochs takes 11.51684618 seconds
epoc 0: 9111.85
100 epochs takes 11.5236680508 seconds
epoc 100: 2590.8
100 epochs takes 11.5863921642 seconds
epoc 200: 1321.07
100 epochs takes 11.642277956 seconds
epoc 300: 901.515
100 epochs takes 11.6188299656 seconds
epoc 400: 730.788
100 epochs takes 11.588654995 seconds
epoc 500: 637.645
100 epochs takes 11.7546470165 seconds
epoc 600: 576.195
100 epochs takes 11.7699801922 seconds
epoc 700: 528.786
100 epochs takes 11.7428228855 seconds
epoc 800: 484.707
100 epochs takes 12.0463659763 seconds
epoc 900: 430.7
100 epochs takes 11.7655608654 seconds
epoc 1000: 365.675
100 epochs takes 11.9206221104 seconds
epoc 1100: 311.159
100 epochs takes 11.8484351635 seconds
epoc 1200: 272.315
100 epochs takes 11.7809638977 seconds
epoc 1300: 245.158
100 epochs takes 11.9608340263 seconds
epoc 1400: 225.124
100 epochs takes 11.8433690071 seconds

### Compute the embedding on the densest 30 columns.

In [ ]:
max30 = np.sort(np.ravel(colsum), axis=None)[len(colsum) - 30]
sp_data30 = spm[:,colsum >= max30]
data30 = mx.ndarray.sparse.csr_matrix(sp_data30)
print(mx.ndarray.sum(data30, axis=0))
print(data30.shape)

We want to start with SVD and see how well it performs.
One question we need to address is **what is the advantage of autoencoder over SVD**.

In [ ]:
U, s, Vh = sp.sparse.linalg.svds(sp_data30, k=10)
res = np.dot(sp_data30.dot(Vh.T), Vh)
print("svd error: " + str(np.sum(np.square(res - sp_data30))))

In [ ]:
params_linear30=train(data30, 5000, internal_act=None)

In [ ]:
params_linear30=train(data30, 5000, internal_act='sigmoid')

### Compute the embedding on the densest 1000 columns.

In [ ]:
max1000 = np.sort(np.ravel(colsum), axis=None)[len(colsum) - 1000]
sp_data1000 = spm[:,colsum >= max1000]
data1000 = mx.ndarray.sparse.csr_matrix(sp_data1000)
print(mx.ndarray.sum(data1000, axis=0))
print(data1000.shape)

In [ ]:
U, s, Vh = sp.sparse.linalg.svds(sp_data1000, k=100)
res = np.dot(sp_data1000.dot(Vh.T), Vh)
print("svd error: " + str(np.sum(np.square(res - sp_data1000))))

In [ ]:
params_linear1000=train(data1000, num_dims=100, num_epoc=5000, internal_act=None)

In [ ]:
pref_matrix = [[0.9, 0.1], [0.1, 0.9]]
block_sizes = [70, 30]
g = ig.Graph.SBM(100, pref_matrix, block_sizes, directed=True)
sim_spm = g.get_adjacency()